In [12]:
%reload_ext autoreload
%autoreload 2

In [13]:
from tqdm.notebook import tqdm
from result_saver import SaverProvider
provider = SaverProvider() 

In [17]:
from Scratch import metadata_loader

df = metadata_loader(_extract=True)
df = df[~df['job_status'].isin(['JobStatus.DONE', 'JobStatus.ERROR', 'JobStatus.CANCELLED'])]
# df = df[:1000]
df

,creation_date,notebook_name,backend_name,job_id,job_name,job_metadata,tags,meas_level,init_qubits,meas_return,...,extra,optimization_level,code,distance,rounds,logical,layout,descr,num_CNOTS_layers,num_CNOT_layers
2598,2024-03-08 17:24:22.330000+01:00,DEEP_get_IQ_calibration_data_Mumbai,ibmq_mumbai,cqnkqdnyffvg008w0t50,NaN,{},"[Calibration, Deep_circuit]",1.0,True,single,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0
2597,2024-03-08 17:18:53.053000+01:00,DEEP_get_IQ_calibration_data_Mumbai,ibmq_mumbai,cqnkmv3s76n0008atqgg,NaN,{},"[Calibration, Deep_circuit]",1.0,True,single,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0
2596,2024-03-08 11:32:10+01:00,get_IQ_calibration_data_Mumbai,ibmq_mumbai,cqneja9s76n0008at70g,NaN,{},[Calibration],1.0,True,single,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2595,2024-03-08 11:32:03.924000+01:00,get_IQ_calibration_data_Mumbai,ibmq_mumbai,cqnej8sy0sfg0089sh7g,NaN,{},[Calibration],1.0,True,single,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2594,2024-03-08 11:31:15.446000+01:00,DEEP_get_IQ_calibration_data_Mumbai,ibmq_mumbai,cqnehwzc8df0008b4ry0,NaN,{},"[Calibration, Deep_circuit]",1.0,True,single,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603,2023-11-08 18:43:47.327864+01:00,diff_rounds_fixed_dist,ibm_sherbrooke,cn5whmyrmwhg008k4660,NaN,{},[],1.0,True,single,...,NaN,NaN,RepetitionCodeCircuit,40.0,7,0,"_is_hex=True, sherbrooke",diff rounds v2,NaN,NaN
1602,2023-11-08 18:43:39.082752+01:00,diff_rounds_fixed_dist,ibm_nazca,cn5whjyrmwhg008k465g,NaN,{},[],1.0,True,single,...,NaN,NaN,RepetitionCodeCircuit,40.0,7,1,"_is_hex=True, sherbrooke",diff rounds v2,NaN,NaN
1601,2023-11-08 18:43:33.352723+01:00,diff_rounds_fixed_dist,ibm_nazca,cn5whhep1am0008qeae0,NaN,{},[],1.0,True,single,...,NaN,NaN,RepetitionCodeCircuit,40.0,7,0,"_is_hex=True, sherbrooke",diff rounds v2,NaN,NaN
1600,2023-11-08 18:43:25.058145+01:00,diff_rounds_fixed_dist,ibm_cusco,cn5whfdrmwhg008k464g,NaN,{},[],1.0,True,single,...,NaN,NaN,RepetitionCodeCircuit,30.0,7,1,"_is_hex=True, sherbrooke",diff rounds v2,NaN,NaN


In [15]:
from time import sleep

start_idx = 0 # Initialize starting index

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            provider.retrieve_job(job_id, _ignore_running=False, overwrite=True)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            # break  
            sleep(5)


  0%|          | 0/1000 [00:00<?, ?it/s]

12:28:40 Warning: Job ID cqnkqdnyffvg008w0t50 not found in /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
12:29:12 Warning: Job ID cqnkmv3s76n0008atqgg not found in /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
12:29:45 Warning: Job ID cqneja9s76n0008at70g not found in /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft

In [16]:
memory = provider.retrieve_job('cnp4xn60k0p00088wmxg').result().get_memory()
print(memory.shape)

(1, 5556)


# update the metadata with the execution date

In [12]:
from Scratch import update_metadata

df = metadata_loader(_extract=True)
df = df[~df['job_status'].isin(['JobStatus.ERROR', 'JobStatus.CANCELLED'])]

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            ibm_job = provider.retrieve_job(job_id, _ignore_running=False)
            additional_dict = {"execution_date": str(ibm_job.result().date)}
            update_metadata(ibm_job, "None_Backend", additional_dict)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            break  
            

  0%|          | 0/1179 [00:00<?, ?it/s]